In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Descarga de datos
!gdown --id '1vcYj7xDwH38AZ8L2AHp-zdqddK-KeaVI'
!gdown --id '1pyy6OWkNxbLsfaIRqzMD06aGqPTIc_DG'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1vcYj7xDwH38AZ8L2AHp-zdqddK-KeaVI
To: /content/DATA_HLF_MDS_2.csv
100% 477M/477M [00:03<00:00, 142MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1pyy6OWkNxbLsfaIRqzMD06aGqPTIc_DG
To: /content/PRINCIPIOS_ACTIVOS_MDS.xlsx
100% 69.2k/69.2k [00:00<00:00, 40.0MB/s]


In [ ]:
#Al importar los datos estan corridos, se puede ver en Resumen y FormaFrama, donde estan corridos en una posicion hacia abajo
main_db = pd.read_csv('/content/DATA_HLF_MDS_2.csv',sep=',')

In [ ]:
code_db = pd.read_excel('/content/PRINCIPIOS_ACTIVOS_MDS.xlsx')
HLF = code_db.loc[:,['PRINCIPIO_ACTIVO','CODIGO_HLF']]

In [ ]:
HLF2 = HLF['CODIGO_HLF'].str.split(', ')
HLF_list = []
for i in range(len(HLF2)):
  HLF_list = HLF_list + HLF2[i] 
try:
  HLF_list.remove('')
except:
  pass
HLF_name = []
for elem in HLF_list:
  HLF_name = HLF_name + HLF[HLF['CODIGO_HLF'].str.contains(elem)]['PRINCIPIO_ACTIVO'].values.tolist()

In [ ]:
HLF_df = pd.DataFrame()
HLF_df['CODIGO_MEDICAMENTO'] = HLF_list
HLF_df['PRINCIPIO_ACTIVO'] = HLF_name
HLF_df

In [ ]:
df = main_db.join(HLF_df.set_index('CODIGO_MEDICAMENTO'), on='CODIGO_MEDICAMENTO')

In [ ]:
corpus = (df['PRES_DENOMINACION']+ ' ' +df['RESUMEN']).dropna().unique()#.to_numpy(dtype= object)

In [ ]:
#PARA DETECTAR LA POSICION DE ENTIDADES TIPO PRINCIPIO ACTIVO, FORMA FARMACO SE COMPARARAN LOS ELEMENTOS DEL CORPUS CON LAS LISTAS CORRESPONDIENTES.
PA = df['PRINCIPIO_ACTIVO'].dropna().unique()
FF = df['FORMA_FARMA'].dropna().unique()

#PARA DETECTAR PERIODICIDAD DE ADMIN SE DETECTARA LA PALABRA "CADA" Y LA DURACION CON "DURANTE" O "DIARIA"

In [ ]:
def split_rule(x):
  output = np.array([],dtype=object)
  for i in  range(len(x)):
    output = np.append(output,x[i].split())
  return output

In [ ]:
PA1 = np.unique(split_rule(PA))
FF1 = np.unique(split_rule(FF))

In [ ]:
def generate_column(corpus,PA,FF):
  output = np.empty([1744922,2], dtype= object)
  idx = 0
  for i in range(len(corpus)):
    c= corpus[i].split()
    j=0
    while j < len(c):
      if c[j] == 'cada' and c[j+1].isnumeric() :
        try:
          output[idx] = np.array([c[j], 'B-PERIODICITY'],dtype = object)
          output[idx+1] = np.array([c[j+1], 'I-PERIODICITY'],dtype = object)
          output[idx+2] = np.array([c[j+2], 'I-PERIODICITY'],dtype = object)
          idx += 3
          j+=3
        except:
          output[idx] = np.array([c[j], 'O'],dtype = object)
          idx += 1
          j+=1
      elif c[j] == 'durante' and c[j+1].isnumeric() :
        try:
          output[idx] = np.array([c[j], 'B-DURATION'],dtype = object)
          output[idx+1] = np.array([c[j+1], 'I-DURATION'],dtype = object)
          output[idx+2] = np.array([c[j+2], 'I-DURATION'],dtype = object)
          idx += 3
          j+=3
        except:
          output[idx] = np.array([c[j], 'O'],dtype = object)
          idx += 1
          j+=1
      elif c[j].lower() in PA or c[j].upper() in PA :
        output[idx] = np.array([c[j], 'ACTVPRNCP'],dtype = object)
        idx += 1
        j+=1
      elif c[j].lower() in FF or c[j].upper() in FF :
        output[idx] = np.array([c[j], 'ADMIN'],dtype = object)
        idx += 1
        j+=1
      else:
        output[idx] = np.array([c[j], 'O'],dtype = object)
        idx += 1
        j+=1
    output[idx] = np.array(['', ''],dtype = object)
    idx += 1
  return output


In [ ]:
txt = generate_column(corpus,PA1,FF1)

In [ ]:
for i in range(len(txt)):
  if txt[i,0] in ['+','CON','SOLUCIÓN']:
    txt[i,1] = txt[i+1,1]

In [ ]:
for i in range(len(txt)):
  if txt[i,1] == 'ADMIN' and txt[i-1,1] == 'B-ADMIN':
    txt[i,1] = 'I-ADMIN'
  elif txt[i,1] == 'ADMIN':
    txt[i,1] = 'B-ADMIN'
  elif txt[i,1] == 'ACTVPRNCP' and txt[i-1,1] == 'B-ACTVPRNCP':
    txt[i,1] = 'I-ACTVPRNCP'
  elif txt[i,1] == 'ACTVPRNCP':
    txt[i,1] = 'B-ACTVPRNCP'
  else:
    pass

In [ ]:
np.savetxt('corpus_recetas.txt',txt,fmt="%s")